## Catio distribution across sites in LTA

This notebook detects the location of the cation across many configurations in a RASPA output PDB file and shows the average vallue across the different sites. The locations of the different sites are defined based on the type of the cages between which they exist. There are three types of pockets in LTA

- The sodalite cage
- Supercage
- The cubical cubical pocket.

Between which there exist three types of sites as shown in the figure below. 

![LTA sites](./structure-siting.jpg)

Please refer to the original paper for more details on the algorithm and context

[In Silico Engineering of Ion-Exchanged Zeolites for High-Performance Carbon Capture in Psa Processes](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4306028)


Replace the Movie file in the repo with yours to detect the cation distribution in your LTA configurations.

## Make some imports

In [1]:
import sys
import os
import numpy as np 
from useful_ import *




## Read in the structure of LTA

Reading in the structure of LTA that is consistent with the simulation box. Use `ase.visualize.view` to view the structure.

In [2]:
# import mofography as mgr
from ase.io import read
mat_atoms = read('./LTA96.cif')

c:\Users\Public\anaconda3\envs\jl_env\lib\site-packages\ase\io\cif.py:401: UserWarning: crystal system 'triclinic' is not interpreted for space group Spacegroup(1, setting=1). This may result in wrong setting!
  warnings.warn(


## Compute the distance grid

Here we are computing a 0.25 A distance grid on the LTA unit cell

In [3]:
spacing=0.25
# import mofography as mgr
from ase.data import covalent_radii 

radii = covalent_radii[mat_atoms.get_atomic_numbers()] 
grid = dgrid_from_atoms(mat_atoms, radii=radii, spacing =spacing, block_size=20000)


Output()

c:\Users\Public\anaconda3\envs\jl_env\lib\site-packages\ase\utils\__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)


## Compute the region segmentation

Breaking down the void volume into pockets with a really thin mask over the framework atoms

In [4]:
regions, maxima = regions_from_dgrid(grid.compute(), mask_thickness=0.5)


c:\Users\Arun Gopalan\OneDrive - The University of Manchester\my_cloned_repos\Insilico-engg-ion-exch-zeolites-2023-ZD-AG-LS\useful_.py:298: UserWarning: Some maxima excluded as they fell inside the mask_thickness
  warnings.warn('Some maxima excluded as they fell inside the mask_thickness')


## Detect connections and windows
Detect the connectivity of pockets and the windows between them in LTA

In [5]:
#* Find the windows and connected regions including the windows on the walls
# connections = mgr.connections_from_regions(dgrid = grid.compute(), region_labels=regions, maxima=maxima, ase_atoms=mat_updated)
rag = get_rag_from_regions_and_grid(regions, grid.compute(),mat_atoms, maxima)
connections = get_connections_for_rag(rag, regions, maxima,mat_atoms, grid.compute())

Output()

c:\Users\Public\anaconda3\envs\jl_env\lib\site-packages\numpy\core\shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


In [6]:
#%% #* Extract the window coordinates from the previus output, create labels for the 
#* cages and windows based on the cage sizes; cubical, sodalite and supercage (1.44, 3.9, 6.2 )
import pandas as pd
windows = np.vstack([c[3] for c in connections if c[1]]) # pick out the window coordinates from connections
cage_radii = grid.compute()[tuple(maxima.T)]
cage_labels = pd.cut(cage_radii, bins=[0,2,5,7], labels = [ 'cubical','sodalite', 'supercage'])
connected_cage_labels = [cage_labels[np.array(c[0])-1] for c in connections if c[1]]
window_labels2 = np.array(['-'.join((c[0],c[1])) for c in connected_cage_labels])
window_labels1 = np.array(['-'.join((str(c[0][0]),str(c[0][1]))) for c in connections if c[1]])
window_labels3 = np.array([c[2] for c in connections if c[1]])
window_labels = [w1+"<br>"+w2+'<br>'+w3 for w1,w2,w3 in zip(window_labels1, window_labels2, window_labels3)]



## Read raspa output PDB

- Read the location of the cations
- Create an 3D interpolator for the identity of the regions from segmenation

In [7]:
# cations = mgr.read_raspa_pdb('./Movie_LTA96_1.1.1_298.150000_0.000000_component_K_0.pdb')
cations = read_raspa_pdb('./Movie_LTA_my_1.1.1_303.000000_0.000000_component_Na_1.pdb')
# cations = mgr.read_raspa_pdb('./Movie_NaX1.0newmodel2_1.1.1_298.150000_100000.000000_component_Na_0.pdb')

# Create an linear interpolator for the regions for assigning arbitrary points (indices) to regions
interpolator = interpolate_labels(regions)
# for config in cations['coords']:
#     interpolator(mgr.get_fractional_coordinates(config, mat_atoms))

## Assign cations to sites

In [8]:
index = 500
ion_region_labels = [str(x) for x in interpolator(get_fractional_coordinates(cations['coords'][index], ase_atoms=mat_atoms)).astype(int)]


In [9]:
def nearest_window_and_distance(coords, ase_atoms, windows, window_labels=None, cut_off=1.0, n_neighbors=4):

    ab = AABB_on_atoms(ase_atoms, points=windows)
    nls = ab.query(coords, query_args=dict(mode='nearest', exclude_ii=True, num_neighbors = n_neighbors )).toNeighborList(sort_by_distance=True)
    point_indices = nls.point_indices.reshape(-1,n_neighbors)
    distances = nls.distances.reshape(-1, n_neighbors)
    if window_labels is None:
        window_labels = list(range(len(windows)))   
    return np.vstack([[window_labels[i], d] if d <= cut_off else ['not_close_enough', d] for i,d in zip(point_indices[:,0], distances[:,0]) ]).astype('object')


## Assign cations ot nearest windows

In [10]:
# this is for all the cations in all the snapshots
labeled_ions =nearest_window_and_distance(np.vstack(cations['coords']), ase_atoms=mat_atoms, windows=windows, window_labels=window_labels, cut_off=1.5)



## Print the average cation fraction at each site

Cations are assigned ot the nearest window based on the types of the adjoining cages, only if the distance to the window is less than 1 A. otherwise, it is said to be `not_close_enough`. 

In [11]:
#* Print out the important ion fractions
print('Fraction too far from windows: {0}'.format(np.mean(np.mean(labeled_ions[:,0]=='not_close_enough'))))
#* Fraction of ions near the supercage-supercage windows
print('Fraction near supercage-supercage windows: {0}'.format(np.mean([('supercage-supercage' in l) for l in labeled_ions[:,0]])))
print('Fraction near sodalite-supercage windows: {0}'.format(np.mean([np.logical_or('sodalite-supercage' in l, 'supercage-sodalite' in l) for l in labeled_ions[:,0]])))
print('Fraction near sodalite-cubical windows: {0}'.format(np.mean([np.logical_or('sodalite-cubical' in l, 'cubical-sodalite' in l) for l in labeled_ions[:,0]])))
print('Fraction near supercage-cubical windows: {0}'.format(np.mean([np.logical_or('supercage-cubical' in l, 'cubical-supercage' in l) for l in labeled_ions[:,0]])))
# np.mean([('supercage-supercage' in l) for l in labeled_ions[:,0]])


Fraction too far from windows: 0.13234375
Fraction near supercage-supercage windows: 0.18498263888888888
Fraction near sodalite-supercage windows: 0.6657465277777778
Fraction near sodalite-cubical windows: 0.0
Fraction near supercage-cubical windows: 0.016927083333333332
